In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 18.5 MB/s 
     |████████████████████████████████| 880 kB 53.3 MB/s 
     |████████████████████████████████| 77 kB 3.1 MB/s 
     |████████████████████████████████| 6.6 MB 49.5 MB/s 
     |████████████████████████████████| 596 kB 50.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e2fcb6a882af897d3500443778ffa18eb39fc5f9fab4d952b913550062e67e52
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!git clone https://github.com/gdunlevy/ENLP-Final-Project.git

Cloning into 'ENLP-Final-Project'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 248 (delta 47), reused 13 (delta 4), pack-reused 155
Receiving objects: 100% (248/248), 1.31 MiB | 23.90 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [3]:
%cd ENLP-Final-Project/
%ls

/content/ENLP-Final-Project
 hetero_cnn.py        parse_data_trial.py   semeval2017_task7/
 homo_cnn.py          read_gold.py         'subtask 1'/
 parse_data_test.py   README.md            'subtask 2'/


# Subtask 2: Heterographic

In [4]:
import parse_data_test as p
import os

f = 'semeval2017_task7/data/test/subtask2-heterographic-test.xml'

mytree = p.ET.parse(f)
myroot = mytree.getroot()

puns = []
for item in myroot.findall('./text'):
  dict1 = {}
  dict1[item.attrib['id']] = {}
  for child in item:
    idd = child.attrib['id']
    dict1[item.attrib['id']][idd] = child.text
  puns.append(dict1)

print(puns[0])

{'het_1': {'het_1_1': "'", 'het_1_2': "'", 'het_1_3': 'I', 'het_1_4': "'", 'het_1_5': 'm', 'het_1_6': 'halfway', 'het_1_7': 'up', 'het_1_8': 'a', 'het_1_9': 'mountain', 'het_1_10': ',', 'het_1_11': "'", 'het_1_12': "'", 'het_1_13': 'Tom', 'het_1_14': 'alleged', 'het_1_15': '.'}}


## BERT

In [5]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased", output_hidden_states=True).to(device)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


$$soft = \frac{e^x}{\sum{e^x}}$$
$$\log(soft) = x - (x$$

In [ ]:
import torch
import copy
from tqdm import tqdm
m = torch.nn.Softmax(dim=0)

res = []

for i in tqdm(puns):
  for pun in i.values():
    poss = [x for x in pun]
    sent = ' '.join([pun[x] for x in pun])
    tok = tokenizer(sent, return_tensors="pt").to(device)
    orig = copy.deepcopy(tok)
    probs = []
    l = len(tok['input_ids'][0])
    for i in range(1, l - 1):
      old = copy.deepcopy(tok['input_ids'][0][i])
      tok['input_ids'][0][i] = torch.tensor(103)
      # print(tokenizer.decode(tok['input_ids'][0]))

      token_logits = model(**tok).logits
      masked_logits = m(token_logits[0, i, :])

      # maximum = masked_logits[torch.argmax(masked_logits)].item()

      probs.append(masked_logits[old].item())
      # print(tokenizer.decode([old]), probs[-1])
      tok = copy.deepcopy(orig)

    tok = tokenizer.tokenize(sent, return_tensors="pt")
    tok.append('')
    min_score, min_pos = 2, poss[0]
    cur = 1
    pos = -1
    l = len(probs)
    for i in range(min(l // 2, l - 1), l):
      # print(tok[i], probs[i])
      if not tok[i].startswith('##'):
        if cur < min_score and i >= l // 2:
          min_score = cur
          min_pos = pos
        pos += 1
        cur = 1
      cur *= probs[i]
    print(sent, pun[poss[min_pos]])
    res.append(poss[min_pos])

  0%|          | 1/1271 [00:00<08:11,  2.58it/s]

' ' I ' m halfway up a mountain , ' ' Tom alleged . alleged


  0%|          | 2/1271 [00:00<09:01,  2.34it/s]

I ' d like to be a Chinese laborer , said Tom coolly . Chinese


  0%|          | 4/1271 [00:01<05:57,  3.55it/s]

...

' Come see my fishpond ' , she said coyly . fishpond


100%|█████████▉| 1270/1271 [06:01<00:00,  3.79it/s]

When the goat ate a Scrabble set , the letters came out in alphabetical ordure . ordure
I wonder how much an anagram weighs ? weighs


100%|██████████| 1271/1271 [06:02<00:00,  3.51it/s]

' ' I don ' t like this Chardonnay , ' ' Tom whined . Tom


In [ ]:
with open('res2.txt', 'w') as fout:
  for i in res:
    fout.write(f'{i}\n')

In [ ]:
gold = []
with open('/content/ENLP-Final-Project/semeval2017_task7/data/test/subtask2-heterographic-test.gold', 'r') as fin:
  for row in fin:
    gold.append(row.strip().split('\t')[1])
print(gold[:5])

['het_1_14', 'het_2_13', 'het_4_11', 'het_5_5', 'het_7_6']


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(gold, res))

0.5814319433516916


## GPT-2

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2", output_hidden_states=True).to(device)

In [ ]:
import torch
import copy
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

m = torch.nn.Softmax(dim=0)

def gpt(gold, ratio=0, ordered=False, lower=False):
  res = []
  position = 0

  for i in tqdm(puns):
    for pun in i.values():
      if lower:
        for x in pun:
          pun[x] = pun[x].lower()
      poss = [x for x in pun]
      sent = ' '.join([pun[x] for x in pun])
      toks = []
      s = False
      for x in pun:
        subwords = tokenizer.tokenize((' ' if s else '') + pun[x])
        s = True
        for pos, t in enumerate(subwords):
          toks.append([t, pos])

      tok = tokenizer(sent, return_tensors="pt").to(device)
      out = model(**tok)
      # hidden = out.hidden_states[0].cpu().detach().numpy().tolist()[0]
      # print(len(hidden))
      # print(hidden)
      token_logits = out.logits
      probs = []
      l = len(tok['input_ids'][0])
      for k in range(l):
        old = tok['input_ids'][0][k]
        masked_logits = m(token_logits[0, k - 1, :])
        maxi = torch.argmax(masked_logits)
        # maximum = [tokenizer.decode(maxi), masked_logits[maxi].item()]
        probs.append([tokenizer.decode([old]), masked_logits[old].item()])
        # print(tokenizer.decode([old]), probs[-1], maximum)

      mult_scores = []
      cur, cur_str = 1, ''
      pos = 0
      l = len(probs)
      for i in range(l):
        # print(probs[i], toks[i])
        if toks[i][1] == 0:
          mult_scores.append([cur, cur_str, pos])
          pos += 1
          cur = 1
          cur_str = ''
        cur *= probs[i][1]
        cur_str += probs[i][0]
      mult_scores.append([cur, cur_str, pos])

      mini = mult_scores[0]
      l = len(mult_scores)
      # print(mult_scores)
      if not ordered:
        for i in range(min(l - 1, int(ratio * l)), l):
          if mult_scores[i][0] < mini[0]:
            mini = mult_scores[i]
        # print(sent, pun[poss[mini[2] - 1]])
        res.append(poss[mini[2] - 1])
      else:
        mult_scores = mult_scores[min(l - 1, int(ratio * l)):]
        mult_scores.sort()
        ans = max([x[2] for x in mult_scores[:ordered]])
        # print(sent, pun[poss[ans - 1]])
        res.append(poss[ans - 1])

    # print(position, res[-1], gold[position])
    # if res[-1] != gold[position]:
    #   for i in mult_scores:
    #     print(i)
    #   print(res[-1], gold[position])
    #   input()
    position += 1
    
  return res

In [ ]:
for i in range(2, 5):
  print(accuracy_score(gold, gpt(gold, ordered=i)))

100%|██████████| 1271/1271 [00:28<00:00, 45.01it/s]


0.6837136113296617


100%|██████████| 1271/1271 [00:26<00:00, 48.21it/s]


0.6664044059795436


100%|██████████| 1271/1271 [00:26<00:00, 48.56it/s]

0.6042486231313926


In [ ]:
for ratio in [0.2, 0.4, 0.6, 0.8]:
  print(ratio, accuracy_score(gold, gpt(gold, ratio, ordered=2)))

100%|██████████| 1271/1271 [00:25<00:00, 49.06it/s]


0.2 0.6923682140047207


100%|██████████| 1271/1271 [00:25<00:00, 49.89it/s]


0.4 0.6302124311565697


100%|██████████| 1271/1271 [00:25<00:00, 50.41it/s]


0.6 0.4956726986624705


 12%|█▏        | 153/1271 [00:03<00:22, 49.20it/s]


KeyboardInterrupt: ignored

In [ ]:
for ratio in [0.2, 0.4, 0.6, 0.8]:
  print(ratio, accuracy_score(gold, gpt(gold, ratio, lower=True)))

100%|██████████| 1271/1271 [00:24<00:00, 52.26it/s]


0.2 0.5523210070810386


100%|██████████| 1271/1271 [00:24<00:00, 52.53it/s]


0.4 0.6734854445318647


100%|██████████| 1271/1271 [00:24<00:00, 51.68it/s]


0.6 0.7442958300550747


100%|██████████| 1271/1271 [00:24<00:00, 51.98it/s]

0.8 0.6876475216365067


In [ ]:
from sklearn.metrics import accuracy_score

for ratio in [0.5, 0.55, 0.65, 0.7]:
  print(ratio, accuracy_score(gold, gpt(gold, ratio, lower=True)))

100%|██████████| 1271/1271 [00:38<00:00, 33.25it/s]


0.5 0.7151848937844217


100%|██████████| 1271/1271 [00:25<00:00, 49.65it/s]


0.55 0.7348544453186467


100%|██████████| 1271/1271 [00:24<00:00, 50.96it/s]


0.65 0.7513768686073957


100%|██████████| 1271/1271 [00:24<00:00, 51.70it/s]

0.7 0.7498033044846577


In [ ]:
from sklearn.metrics import accuracy_score

for ratio in [0.5, 0.55, 0.65, 0.7]:
  print(ratio, accuracy_score(gold, gpt(gold, ratio)))

100%|██████████| 1271/1271 [00:27<00:00, 46.91it/s]


0.5 0.7340676632572777


100%|██████████| 1271/1271 [00:24<00:00, 50.86it/s]


0.55 0.7513768686073957


100%|██████████| 1271/1271 [00:25<00:00, 49.64it/s]


0.65 0.7828481510621558


100%|██████████| 1271/1271 [00:25<00:00, 49.05it/s]

0.7 0.7836349331235248


# Subtask 2: Homographic

Let's try the same approach, lowest probability word.

In [ ]:
import parse_data_test as p
import os

f = 'semeval2017_task7/data/test/subtask2-homographic-test.xml'

mytree = p.ET.parse(f)
myroot = mytree.getroot()

puns = []
for item in myroot.findall('./text'):
  dict1 = {}
  dict1[item.attrib['id']] = {}
  for child in item:
    idd = child.attrib['id']
    dict1[item.attrib['id']][idd] = child.text
  puns.append(dict1)

print(puns[0])

{'hom_1': {'hom_1_1': 'They', 'hom_1_2': 'hid', 'hom_1_3': 'from', 'hom_1_4': 'the', 'hom_1_5': 'gunman', 'hom_1_6': 'in', 'hom_1_7': 'a', 'hom_1_8': 'sauna', 'hom_1_9': 'where', 'hom_1_10': 'they', 'hom_1_11': 'could', 'hom_1_12': 'sweat', 'hom_1_13': 'it', 'hom_1_14': 'out', 'hom_1_15': '.'}}


In [ ]:
gold = []
with open('/content/ENLP-Final-Project/semeval2017_task7/data/test/subtask2-homographic-test.gold', 'r') as fin:
  for row in fin:
    gold.append(row.strip().split('\t')[1])
print(gold[:5])

['hom_1_12', 'hom_2_9', 'hom_3_7', 'hom_4_5', 'hom_5_15']


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2", output_hidden_states=True).to(device)

In [ ]:
from sklearn.metrics import accuracy_score

for ratio in [0.0, 0.2, 0.4, 0.6, 0.8]:
  print(ratio, accuracy_score(gold, gpt(gold, ratio)))

100%|██████████| 1607/1607 [00:31<00:00, 51.12it/s]


0.0 0.10267579340385811


100%|██████████| 1607/1607 [00:30<00:00, 53.10it/s]


0.2 0.1281891723708774


100%|██████████| 1607/1607 [00:31<00:00, 51.50it/s]


0.4 0.2762912258867455


100%|██████████| 1607/1607 [00:30<00:00, 52.24it/s]


0.6 0.41008089607965154


100%|██████████| 1607/1607 [00:30<00:00, 52.25it/s]

0.8 0.42501555693839455


In [ ]:
print(1.0, accuracy_score(gold, gpt(1.0)))

100%|██████████| 1607/1607 [00:31<00:00, 51.57it/s]

1.0 0.4399502177971375
